In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from datetime import datetime
import pickle
import os
import json
import seaborn as sns
import math
import torch.nn.functional as F


In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
class FeatureCorrelationModule(nn.Module):
    """
    A module that explicitly models feature correlations to better handle MNAR scenarios.
    """
    def __init__(self, num_features, d_model, dropout=0.1):
        super().__init__()
        self.correlation_proj = nn.Linear(d_model, d_model)
        self.feature_gate = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.Sigmoid()
        )
        self.correlation_norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        """
        Args:
            x: Input features [batch_size, num_features, d_model]
            mask: Missing value mask [batch_size, num_features]
            
        Returns:
            Correlation-enhanced features
        """
        batch_size, num_features, d_model = x.size()
        
        # Project features for correlation computation
        x_proj = self.correlation_proj(x)
        
        # Compute pairwise feature correlations (scaled dot-product)
        corr_matrix = torch.bmm(x_proj, x_proj.transpose(1, 2)) / math.sqrt(d_model)
        
        # If we have a mask, adjust correlation for missing values
        if mask is not None:
            # Create attention mask (1 for observed values, 0 for missing)
            obs_mask = 1 - mask.float()
            mask_matrix = torch.bmm(obs_mask.unsqueeze(2), obs_mask.unsqueeze(1))
            
            # Apply mask to correlation matrix (masked positions get -1e9)
            masked_corr = corr_matrix.masked_fill(mask_matrix == 0, -1e9)
            
            # Softmax to get normalized correlation weights
            corr_weights = F.softmax(masked_corr, dim=-1)
        else:
            corr_weights = F.softmax(corr_matrix, dim=-1)
        
        # Apply correlation weights to propagate information across features
        corr_features = torch.bmm(corr_weights, x)
        
        # Compute feature-specific gates to control information flow
        gates = self.feature_gate(x)
        
        # Gate the correlation features and apply residual connection
        gated_corr = gates * corr_features
        enhanced_features = x + self.dropout(gated_corr)
        
        # Apply layer normalization
        enhanced_features = self.correlation_norm(enhanced_features)
        
        return enhanced_features

In [ ]:
class FeatureValueDependentEncoder(nn.Module):
    """
    An enhanced module that explicitly models the relationship between feature values and missingness,
    which is crucial for MNAR scenarios.
    """
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        # Deep value encoder with skip connections
        self.value_encoder_1 = nn.Linear(d_model, d_model * 2)
        self.value_encoder_2 = nn.Linear(d_model * 2, d_model * 2)
        self.value_encoder_3 = nn.Linear(d_model * 2, d_model)
        
        self.value_norm_1 = nn.LayerNorm(d_model * 2)
        self.value_norm_2 = nn.LayerNorm(d_model * 2)
        self.value_norm_3 = nn.LayerNorm(d_model)
        
        # Enhanced missingness encoder with more capacity
        self.missingness_encoder_1 = nn.Linear(1, d_model)
        self.missingness_encoder_2 = nn.Linear(d_model, d_model)
        self.missingness_norm = nn.LayerNorm(d_model)
        
        # Multi-head attention for better fusion
        self.attention = nn.MultiheadAttention(d_model, num_heads=8, dropout=dropout)
        
        # Gated cross-attention for value-missingness interaction
        self.cross_gate = nn.Sequential(
            nn.Linear(d_model * 2, d_model),
            nn.Sigmoid()
        )
        
        # Enhanced fusion layer
        self.fusion_layer = nn.Sequential(
            nn.Linear(d_model * 2, d_model * 2),
            nn.GELU(),
            nn.Dropout(dropout * 1.5),  # Increased dropout
            nn.Linear(d_model * 2, d_model),
            nn.LayerNorm(d_model)
        )
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        """
        Args:
            x: Input features [batch_size, num_features, d_model]
            mask: Missing value mask [batch_size, num_features]
            
        Returns:
            Features with enhanced MNAR understanding
        """
        # Encode feature values with residual connections
        value_enc_1 = self.value_norm_1(F.gelu(self.value_encoder_1(x)))
        value_enc_2 = self.value_norm_2(F.gelu(self.value_encoder_2(value_enc_1)) + value_enc_1)
        value_encoding = self.value_norm_3(F.gelu(self.value_encoder_3(value_enc_2)))
        
        # If we have a mask, encode missingness patterns
        if mask is not None:
            # Encode missingness with non-linear transformations
            mask_enc_1 = F.gelu(self.missingness_encoder_1(mask.float().unsqueeze(-1)))
            mask_encoding = self.missingness_norm(F.gelu(self.missingness_encoder_2(mask_enc_1)))
            
            # Reshape for attention: [seq_len, batch_size, d_model]
            batch_size, num_features, d_model = value_encoding.size()
            v_enc = value_encoding.transpose(0, 1)
            m_enc = mask_encoding.transpose(0, 1)
            
            # Self-attention on value encodings with mask as query
            attn_output, attn_weights = self.attention(m_enc, v_enc, v_enc)
            
            # Reshape back: [batch_size, seq_len, d_model]
            attn_output = attn_output.transpose(0, 1)
            
            # Feature-specific gating based on value and missingness
            combined_for_gate = torch.cat([mask_encoding, attn_output], dim=-1)
            gate = self.cross_gate(combined_for_gate)
            
            # Gated combination
            gated_attn = gate * attn_output
            
            # Enhanced mask encoding with attention output
            mask_encoding = mask_encoding + self.dropout(gated_attn)
            
            # Concatenate value and enhanced missingness encodings
            combined = torch.cat([value_encoding, mask_encoding], dim=-1)
            enhanced = self.fusion_layer(combined)
            
            # Add residual connection
            enhanced = enhanced + value_encoding
        else:
            # Without mask, just use value encoding
            enhanced = value_encoding
            
        return enhanced

In [ ]:
class RelativePositionEncoding(nn.Module):
    """
    Learnable relative position encoding for features in the transformer model.
    This allows the model to understand relationships between features based on their positions.
    """
    def __init__(self, max_seq_len, d_model):
        super().__init__()
        self.rel_pos_embedding = nn.Parameter(torch.randn(2 * max_seq_len - 1, d_model))
        self.max_seq_len = max_seq_len
        
    def forward(self, x):
        """
        Apply relative positional encodings to the input.
        
        Args:
            x: Input tensor [batch_size, seq_len, d_model]
            
        Returns:
            Tensor with relative positional information.
        """
        seq_len = x.size(1)
        pos_indices = torch.arange(seq_len, device=x.device)
        rel_pos_indices = pos_indices.unsqueeze(1) - pos_indices.unsqueeze(0) + self.max_seq_len - 1
        rel_pos_encoded = self.rel_pos_embedding[rel_pos_indices]
        
        return rel_pos_encoded

In [ ]:
class MultiHeadAttentionWithRelPos(nn.Module):
    """
    Multi-head attention with relative positional encoding.
    """
    def __init__(self, d_model, num_heads, dropout=0.1, max_seq_len=1000):
        super().__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        
        # Linear projections for Q, K, V
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.v_proj = nn.Linear(d_model, d_model)
        self.out_proj = nn.Linear(d_model, d_model)
        
        self.rel_pos_encoding = RelativePositionEncoding(max_seq_len, d_model)
        
        self.rel_pos_proj = nn.Linear(d_model, d_model)

        self.scale = self.head_dim ** -0.5
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, query, key, value, key_padding_mask=None, need_weights=False):
        """
        Forward pass with relative positional encoding.
        
        Args:
            query, key, value: Input tensors [batch_size, seq_len, d_model]
            key_padding_mask: Mask for padded values [batch_size, seq_len]
            need_weights: Whether to return attention weights
            
        Returns:
            Output tensor and optionally attention weights
        """
        batch_size = query.size(0)
        seq_len = query.size(1)
        
        #Linear projections and reshape for multi-head attention
        q = self.q_proj(query).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_proj(key).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_proj(value).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        #content-based attention scores
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) * self.scale  # [batch, heads, seq_len, seq_len]
        
        rel_bias = torch.zeros((seq_len, seq_len), device=query.device)
        positions = torch.arange(seq_len, device=query.device)
        relative_positions = positions.unsqueeze(1) - positions.unsqueeze(0)

        rel_bias = -torch.abs(relative_positions) * 0.1

        attn_scores = attn_scores + rel_bias.unsqueeze(0).unsqueeze(0)

        if key_padding_mask is not None:
            attn_mask = key_padding_mask.unsqueeze(1).unsqueeze(2)
            attn_scores = attn_scores.masked_fill(attn_mask, float('-inf'))

        attn_weights = torch.softmax(attn_scores, dim=-1)
        attn_weights = self.dropout(attn_weights)

        output = torch.matmul(attn_weights, v)
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)

        output = self.out_proj(output)
        
        if need_weights:
            return output, attn_weights
        else:
            return output


In [ ]:
class RelativePositionTransformerLayer(nn.Module):
    """
    Transformer encoder layer with relative positional encoding.
    """
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, 
                 activation="gelu", max_seq_len=1000, norm_first=True):
        super().__init__()

        self.self_attn = MultiHeadAttentionWithRelPos(
            d_model, nhead, dropout=dropout, max_seq_len=max_seq_len
        )

        #FFN
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        
        #Activation
        self.activation = getattr(nn.functional, activation)
        self.norm_first = norm_first
        
    def forward(self, src, src_key_padding_mask=None):
        """
        Forward pass of the transformer layer.
        
        Args:
            src: Input tensor [batch_size, seq_len, d_model]
            src_key_padding_mask: Mask for padded values [batch_size, seq_len]
            
        Returns:
            Processed tensor
        """
        # Pre-norm architecture
        if self.norm_first:
            src2 = self.norm1(src)
            src2 = self.self_attn(src2, src2, src2, key_padding_mask=src_key_padding_mask)
            src = src + self.dropout1(src2)
            
            src2 = self.norm2(src)
            src2 = self.linear2(self.dropout(self.activation(self.linear1(src2))))
            src = src + self.dropout2(src2)
        else:
            # Multi-head attention block with post-normalization
            src2 = self.self_attn(src, src, src, key_padding_mask=src_key_padding_mask)
            src = self.norm1(src + self.dropout1(src2))
            
            # Feedforward block with post-normalization
            src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
            src = self.norm2(src + self.dropout2(src2))
            
        return src

In [ ]:
class RelativePositionTransformerEncoder(nn.Module):
    """
    Transformer encoder with relative positional encoding.
    """
    def __init__(self, encoder_layer, num_layers):
        super().__init__()
        self.layers = nn.ModuleList([encoder_layer for _ in range(num_layers)])
        self.num_layers = num_layers
        
    def forward(self, src, mask=None):
        """
        Forward pass of the transformer encoder.
        
        Args:
            src: Input tensor [batch_size, seq_len, d_model]
            mask: Mask for padded values [batch_size, seq_len]
            
        Returns:
            Encoded tensor
        """
        output = src
        for layer in self.layers:
            output = layer(output, src_key_padding_mask=mask)
        return output

In [ ]:
class TabularTransformerWithRelPos(nn.Module):
    """
    Enhanced transformer model for tabular data imputation with improved
    MNAR handling through correlation modeling.
    """
    def __init__(self, 
                 num_features, 
                 d_model=128, 
                 nhead=8, 
                 num_layers=3, 
                 dim_feedforward=512, 
                 dropout=0.1, 
                 activation='gelu',
                 max_seq_len=1000):
        super().__init__()
        
        self.d_model = d_model
        self.num_features = num_features
        
        # Feature value embedding
        self.value_embedding = nn.Sequential(
            nn.Linear(1, d_model),
            nn.LayerNorm(d_model),
            nn.Dropout(dropout * 0.5),
            nn.GELU(),
            nn.Linear(d_model, d_model),
            nn.LayerNorm(d_model)
        )
        
        # Column embedding (learnable)
        self.column_embedding = nn.Embedding(num_features, d_model)
        
        # Missing value embedding
        self.missing_embedding = nn.Parameter(torch.randn(1, d_model))
        
        # Feature correlation module to model dependencies between features
        self.feature_correlation = FeatureCorrelationModule(num_features, d_model, dropout)
        
        # Feature-value dependent encoder for MNAR awareness
        self.feature_value_encoder = FeatureValueDependentEncoder(d_model, dropout)
        
        # Layer normalization before transformer
        self.norm = nn.LayerNorm(d_model)
        
        # Create transformer encoder layers
        encoder_layer = RelativePositionTransformerLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation=activation,
            max_seq_len=max_seq_len,
            norm_first=True
        )
        
        # Create transformer encoder
        self.transformer_encoder = RelativePositionTransformerEncoder(encoder_layer, num_layers)

        # Output projection
        self.output_projection = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.LayerNorm(d_model),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, d_model // 2),
            nn.GELU(),
            nn.Linear(d_model // 2, 1)
        )
        
        # Initialize weights
        self._init_weights()
        
    def _init_weights(self):
        """Initialize weights using Kaiming initialization for better convergence"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Embedding):
                nn.init.normal_(m.weight, mean=0.0, std=0.02)
                
    def _generate_attention_mask(self, mask):
        """Generate attention mask for transformer"""
        if mask is None:
            return None
        attn_mask = mask.bool()
        return attn_mask
                
    def forward(self, x, column_indices, mask=None):
        """
        Forward pass with enhanced correlation modeling for MNAR patterns.
        
        Args:
            x: Input tensor [batch_size, num_features]
            column_indices: Tensor of column indices [num_features]
            mask: Optional mask for missing values [batch_size, num_features]
            
        Returns:
            Tensor of predicted values [batch_size, num_features]
        """
        batch_size = x.size(0)
        
        # Reshape to [batch_size, num_features, 1] for embedding
        x = x.unsqueeze(-1)
        
        # Embed feature values
        x_embedded = self.value_embedding(x)
        
        # Add column embeddings
        col_embed = self.column_embedding(column_indices).unsqueeze(0).expand(batch_size, -1, -1)
        x_embedded = x_embedded + col_embed
        
        # Apply special handling for missing values
        if mask is not None:
            # Expand mask for broadcasting
            mask_expanded = mask.unsqueeze(-1).expand_as(x_embedded)
            # Replace embeddings for missing values with learned missing embedding
            missing_embed = self.missing_embedding.expand_as(x_embedded)
            x_embedded = torch.where(mask_expanded == 1, missing_embed, x_embedded)
        
        # Enhanced feature correlation modeling - key for MNAR patterns
        x_correlated = self.feature_correlation(x_embedded, mask)
        
        # Feature-value dependent encoding - explicitly models relationship 
        # between values and missingness
        x_value_aware = self.feature_value_encoder(x_correlated, mask)
        
        # Apply layer normalization
        x_embedded = self.norm(x_value_aware)
        
        # Apply transformer encoder
        attn_mask = self._generate_attention_mask(mask) if mask is not None else None
        x_encoded = self.transformer_encoder(x_embedded, attn_mask)
        
        # Project to output
        output = self.output_projection(x_encoded).squeeze(-1)
        
        return output

In [ ]:
class EnsembleModel(nn.Module):
    """
    Ensemble of transformer models for improved prediction.
    """
    def __init__(self, num_features, config, num_models=3):
        super().__init__()
        self.num_models = num_models
        
        # Create multiple base models
        self.models = nn.ModuleList([
            TabularTransformerWithRelPos(
                num_features=num_features,
                d_model=config["d_model"],
                nhead=config["num_heads"],
                num_layers=config["num_layers"],
                dim_feedforward=config["dim_feedforward"],
                dropout=config["dropout"],
                activation=config["activation"],
                max_seq_len=max(2 * num_features, 100)
            ) for _ in range(num_models)
        ])
        
    def forward(self, x, column_indices, mask=None):
        # Get predictions from all models
        all_preds = []
        for model in self.models:
            preds = model(x, column_indices, mask)
            all_preds.append(preds.unsqueeze(0))
        
        # Stack and average predictions
        all_preds = torch.cat(all_preds, dim=0)
        avg_preds = torch.mean(all_preds, dim=0)
        
        return avg_preds

In [ ]:
def create_missing_mask(data, missing_fraction=0.2, mechanism="MCAR"):
    """
    Create a mask for missing values using different mechanisms, with enhanced MAR modeling.
    
    Args:
        data (torch.Tensor): Input data tensor
        missing_fraction (float): Fraction of values to mask
        mechanism (str): One of "MCAR", "MAR", or "MNAR"
        
    Returns:
        torch.Tensor: Binary mask (1 = missing, 0 = present)
    """
    try:
        # MCAR implementation - completely random
        if mechanism == "MCAR":
            mask = torch.rand(data.shape, device=data.device) < missing_fraction
            return mask.int()
        
        # Simulated MAR (missing at random) implementation with enhanced realism
        # In MAR, missingness depends on observed values but not on missing values
        elif mechanism == "MAR":
            # Create empty mask
            mask = torch.zeros(data.shape, device=data.device, dtype=torch.int)
            
            # Number of features
            num_features = data.shape[1]
            batch_size = data.shape[0]
            
            # Feature dependencies - which features influence the missingness of others
            # In real data, missing patterns often form groups or clusters
            
            # Create feature groups (simulation of related features)
            num_groups = min(5, num_features // 2)
            features_per_group = num_features // num_groups
            
            for group_idx in range(num_groups):
                # Get features in this group
                start_idx = group_idx * features_per_group
                end_idx = min(start_idx + features_per_group, num_features)
                group_features = list(range(start_idx, end_idx))
                
                if len(group_features) < 2:
                    continue  # Skip too small groups
                
                # For each feature in the group, select 1-3 other features from the group as predictors
                for feature_idx in group_features:
                    # Choose predictor features from the same group but excluding the current feature
                    other_features = [f for f in group_features if f != feature_idx]
                    
                    # Determine how many predictors to use based on group size
                    num_predictors = min(3, len(other_features))
                    
                    if num_predictors == 0:
                        # Fallback to MCAR for this feature if no predictors available
                        mask[:, feature_idx] = (torch.rand(batch_size, device=data.device) < missing_fraction).int()
                        continue
                    
                    # Randomly select predictor features
                    predictor_indices = torch.tensor(other_features, device=data.device)[
                        torch.randperm(len(other_features), device=data.device)[:num_predictors]
                    ]
                    
                    # Compute the prediction function - more complex than simple sum
                    # Use a combination of sum, product, and max to create more realistic dependencies
                    pred_values = torch.zeros(batch_size, device=data.device)
                    
                    for i, p_idx in enumerate(predictor_indices):
                        if i == 0:
                            # First predictor is base
                            pred_values = data[:, p_idx]
                        elif i == 1:
                            # Second predictor creates interaction effect
                            pred_values = pred_values * (1 + 0.5 * data[:, p_idx])
                        else:
                            # Third+ predictor has conditional effect
                            pred_values = pred_values + 0.3 * torch.where(
                                data[:, p_idx] > 0, 
                                data[:, p_idx], 
                                torch.zeros_like(data[:, p_idx])
                            )
                    
                    # Normalize to [0, 1] range
                    if pred_values.max() > pred_values.min():
                        normalized_values = (pred_values - pred_values.min()) / (pred_values.max() - pred_values.min())
                    else:
                        normalized_values = torch.zeros_like(pred_values)
                    
                    # Apply non-linear transformation with asymmetric threshold
                    # Create a more realistic pattern with step function and noise
                    
                    # Step function with noise
                    noise = torch.rand(batch_size, device=data.device) * 0.3  # Random noise
                    
                    # Apply threshold based on normalized values and missing fraction
                    # Calculate adaptive threshold to maintain overall missing rate
                    sorted_values, _ = torch.sort(normalized_values + noise)
                    threshold_idx = int((1 - missing_fraction) * batch_size)
                    if threshold_idx < batch_size:
                        threshold = sorted_values[threshold_idx]
                    else:
                        threshold = 1.1  # Ensures no missing values if threshold_idx is out of range
                    
                    # Create binary mask
                    mask[:, feature_idx] = (normalized_values + noise > threshold).int()
            
            # Fill any remaining features with MCAR
            for feature_idx in range(num_features):
                if mask[:, feature_idx].sum() == 0:  # No missing values assigned yet
                    mask[:, feature_idx] = (torch.rand(batch_size, device=data.device) < missing_fraction).int()
            
            # Ensure we have the correct overall missing rate
            actual_missing_rate = mask.float().mean().item()
            
            # Adjust if needed
            if abs(actual_missing_rate - missing_fraction) > 0.05:
                # Too much difference, adjust by adding/removing some random missing values
                if actual_missing_rate < missing_fraction:
                    # Need to add more missing values
                    additional_mask = torch.rand(data.shape, device=data.device) < (missing_fraction - actual_missing_rate) * 2
                    additional_mask = additional_mask.int() & (1 - mask)  # Only add where not already missing
                    mask = mask | additional_mask
                else:
                    # Need to remove some missing values
                    removal_mask = torch.rand(data.shape, device=data.device) < (actual_missing_rate - missing_fraction) * 2
                    removal_mask = removal_mask.int() & mask  # Only remove from missing values
                    mask = mask & (1 - removal_mask)
            
            return mask
        
        # Simulated MNAR (missing not at random) implementation
        # In MNAR, missingness depends on the missing values themselves
        elif mechanism == "MNAR":
            # Create a base random mask
            mask = torch.zeros(data.shape, device=data.device, dtype=torch.int)
            
            # Number of features
            num_features = data.shape[1]
            batch_size = data.shape[0]
            
            # Group features for more realistic patterns
            # Some features may have MNAR patterns based on their own values
            # Others may have MNAR patterns based on related features
            
            # Determine pattern type for each feature (0: own value, 1: extreme values, 2: related feature)
            pattern_types = torch.randint(0, 3, (num_features,), device=data.device)
            
            for col_idx in range(num_features):
                pattern_type = pattern_types[col_idx].item()
                
                if pattern_type == 0:
                    # Pattern based on own value - higher values more likely to be missing
                    # This is a common MNAR pattern in medical data
                    col_values = data[:, col_idx]
                    
                    # Normalize to [0, 1]
                    if col_values.max() > col_values.min():
                        normalized_values = (col_values - col_values.min()) / (col_values.max() - col_values.min())
                    else:
                        normalized_values = torch.zeros_like(col_values)
                    
                    # Higher values have higher probability of being missing
                    # Add noise for more realistic pattern
                    prob = normalized_values * 0.8 + torch.rand(batch_size, device=data.device) * 0.2
                    
                elif pattern_type == 1:
                    # Pattern based on extreme values (both high and low)
                    # This is common in censored data (values outside measurement range)
                    col_values = data[:, col_idx]
                    
                    # Normalize to [-1, 1] centered at 0
                    if col_values.max() > col_values.min():
                        normalized_values = 2 * (col_values - col_values.min()) / (col_values.max() - col_values.min()) - 1
                    else:
                        normalized_values = torch.zeros_like(col_values)
                    
                    # Extreme values (away from center) more likely to be missing
                    extremeness = torch.abs(normalized_values)
                    prob = extremeness * 0.7 + torch.rand(batch_size, device=data.device) * 0.3
                    
                else:
                    # Pattern based on related feature (still MNAR but more complex)
                    # Choose random other feature
                    other_idx = (col_idx + torch.randint(1, num_features, (1,), device=data.device).item()) % num_features
                    
                    # Use combination of own value and other feature
                    own_values = data[:, col_idx]
                    other_values = data[:, other_idx]
                    
                    # Normalize both
                    if own_values.max() > own_values.min():
                        norm_own = (own_values - own_values.min()) / (own_values.max() - own_values.min())
                    else:
                        norm_own = torch.zeros_like(own_values)
                        
                    if other_values.max() > other_values.min():
                        norm_other = (other_values - other_values.min()) / (other_values.max() - other_values.min())
                    else:
                        norm_other = torch.zeros_like(other_values)
                    
                    # Complex interaction: missing when both values are high or both are low
                    interaction = 1 - 2 * torch.abs(norm_own - norm_other)  # High when similar, low when different
                    prob = interaction * 0.7 + torch.rand(batch_size, device=data.device) * 0.3
                
                # Apply threshold to maintain missing rate
                sorted_probs, _ = torch.sort(prob)
                threshold_idx = int((1 - missing_fraction) * batch_size)
                if threshold_idx < batch_size:
                    threshold = sorted_probs[threshold_idx]
                else:
                    threshold = 1.1  # Ensures no missing values if threshold_idx is out of range
                
                # Set mask
                mask[:, col_idx] = (prob > threshold).int()
            
            return mask
        
        # Default to MCAR if unknown mechanism
        else:
            print(f"Unknown missing data mechanism: {mechanism}. Defaulting to MCAR.")
            return create_missing_mask(data, missing_fraction, "MCAR")
            
    except Exception as e:
        print(f"Error creating {mechanism} mask: {e}")
        # Fall back to MCAR if there's an error
        return create_missing_mask(data, missing_fraction, "MCAR")

In [ ]:
def load_and_prepare_data(data_path, test_size=0.2, val_size=0.1, random_state=SEED):
    """
    Load and prepare data for model training.
    
    Args:
        data_path (str): Path to the CSV file
        test_size (float): Proportion of data to use for testing
        val_size (float): Proportion of training data to use for validation
        random_state (int): Random seed for reproducibility
        
    Returns:
        tuple: Preprocessed train, validation, test tensors, scaler, and column indices
    """
    print(f"Loading data from {data_path}")
    df = pd.read_csv(data_path, index_col=None)
    
    # Check for missing values
    missing_count = df.isna().sum().sum()
    if missing_count > 0:
        print(f"Warning: Dataset contains {missing_count} missing values. These will be handled in preprocessing.")
        # Simple imputation for missing values
        df = df.fillna(df.mean())
    
    # Convert to numpy for preprocessing
    data = df.to_numpy()
    
    # Split data into train and test
    train_val_data, test_data = train_test_split(
        data, 
        test_size=test_size, 
        random_state=random_state
    )
    
    # Split train_val into train and validation
    val_ratio = val_size / (1 - test_size)
    train_data, val_data = train_test_split(
        train_val_data,
        test_size=val_ratio,
        random_state=random_state
    )
    
    # Normalize data
    scaler = StandardScaler()
    train_data_scaled = scaler.fit_transform(train_data)
    val_data_scaled = scaler.transform(val_data)
    test_data_scaled = scaler.transform(test_data)
    
    # Convert to PyTorch tensors
    train_tensor = torch.tensor(train_data_scaled, dtype=torch.float32).to(device)
    val_tensor = torch.tensor(val_data_scaled, dtype=torch.float32).to(device)
    test_tensor = torch.tensor(test_data_scaled, dtype=torch.float32).to(device)
    
    # Create column indices
    column_indices = torch.arange(train_tensor.shape[1]).to(device)
    
    print(f"Dataset shape: {df.shape}")
    print(f"Number of training samples: {train_tensor.shape[0]}")
    print(f"Number of validation samples: {val_tensor.shape[0]}")
    print(f"Number of test samples: {test_tensor.shape[0]}")
    print(f"Number of features: {train_tensor.shape[1]}")
    
    return train_tensor, val_tensor, test_tensor, scaler, column_indices

In [ ]:
def compute_mnar_weighted_loss(predictions, ground_truth, mask, mechanism=None):
    """
    Enhanced compute loss with adaptive weighting for MNAR and MAR scenarios,
    with special focus on high missing percentage scenarios.
    
    Args:
        predictions (torch.Tensor): Predicted values
        ground_truth (torch.Tensor): True values
        mask (torch.Tensor): Binary mask (1 = missing, 0 = present)
        mechanism (str): Missing data mechanism (MCAR, MAR, MNAR)
        
    Returns:
        torch.Tensor: Weighted loss
    """
    batch_size, num_features = ground_truth.shape
    
    # Use Huber loss for robustness to outliers - with beta adjusted
    huber_loss = nn.SmoothL1Loss(reduction='none', beta=0.15)
    element_loss = huber_loss(predictions, ground_truth)
    
    # Calculate missing percentage in this batch
    batch_missing_pct = mask.float().mean().item()
    
    # Base mechanism-specific weighting with adaptive scaling based on missing percentage
    if mechanism == "MNAR":
        # Higher weight for MNAR as missingness increases
        base_weight = 2.0 + batch_missing_pct * 2.0  # Scales from 2.0 at 0% to 4.0 at 100%
    elif mechanism == "MAR":
        # Significantly higher weight for MAR at high missing percentages
        base_weight = 1.0 + batch_missing_pct * 3.0  # Scales from 1.0 at 0% to 4.0 at 100%
    else:  # MCAR
        # Slightly higher weight for MCAR at higher missing percentages
        base_weight = 1.0 + batch_missing_pct * 0.5  # Scales from 1.0 at 0% to 1.5 at 100%
    
    # Common pattern detection - features that tend to be missing together
    missing_per_feature = mask.float().mean(dim=0)
    missing_correlation = torch.zeros_like(mask, dtype=torch.float32)
    
    # For MAR and MNAR, apply more sophisticated weighting
    if mechanism in ["MAR", "MNAR"]:
        # Get observed feature values
        observed_mask = 1 - mask
        
        # Calculate feature statistics where observed
        feature_means = torch.zeros_like(ground_truth)
        feature_stds = torch.ones_like(ground_truth)
        
        for i in range(num_features):
            feat_observed = ground_truth[:, i] * observed_mask[:, i]
            feat_count = observed_mask[:, i].sum() + 1e-8
            
            # Mean calculation
            feat_mean = feat_observed.sum() / feat_count
            feature_means[:, i] = feat_mean
            
            # Standard deviation calculation
            if feat_count > 1:
                feat_variance = ((feat_observed - feat_mean * observed_mask[:, i]) ** 2).sum() / feat_count
                feat_std = torch.sqrt(feat_variance + 1e-8)
                feature_stds[:, i] = feat_std
        
        # Calculate z-scores (deviation from mean, normalized by std)
        z_scores = torch.abs(ground_truth - feature_means) / (feature_stds + 1e-8)
        
        # Clip to prevent extreme values
        z_scores = torch.clamp(z_scores, 0, 3.5)
        
        if mechanism == "MNAR":
            # For MNAR, higher deviation = higher weight (abnormal values more likely missing)
            # Use sigmoid for smoother weighting, centered around z=1.5
            value_weights = 1.0 + torch.sigmoid((z_scores - 1.5) * 2.0)
            
            # Final weights for MNAR combine base weight with value-dependent weights
            weights = base_weight * mask * value_weights
            
        elif mechanism == "MAR":
            # For MAR, detect feature interdependencies for better weighting
            # Compute an interdependency score for each sample/feature
            for i in range(num_features):
                # Find correlated features (ones that tend to be missing together)
                other_features_mask = mask[:, torch.arange(num_features) != i]
                this_feature_mask = mask[:, i].unsqueeze(1).expand(-1, num_features-1)
                
                # Calculate correlation factor for this feature with others
                correlation_factor = torch.mean((other_features_mask == this_feature_mask).float(), dim=1)
                missing_correlation[:, i] = correlation_factor
            
            # Higher interdependency = higher weight (harder to predict)
            # Scale to reasonable range
            interdep_weights = 1.0 + missing_correlation
            
            # Add position-based weighting for MAR (later features often depend on earlier ones)
            position_weight = torch.ones_like(mask, dtype=torch.float32, device=mask.device)
            for i in range(num_features):
                # Later features get progressively higher weights
                position_weight[:, i] = 1.0 + (i / num_features) * 0.5
            
            # Final weights for MAR with interdependency and position weighting
            weights = base_weight * mask * interdep_weights * position_weight
            
    else:  # MCAR
        # For MCAR, weight by relative position and apply adaptive batch weighting
        position_weight = torch.ones_like(mask, dtype=torch.float32, device=mask.device)
        for i in range(num_features):
            # Middle features get slightly higher weights (typically more informative)
            rel_pos = abs(i - num_features/2) / (num_features/2)  # 0 at middle, 1 at edges
            position_weight[:, i] = 1.0 + (1.0 - rel_pos) * 0.3  # Max 1.3 at middle
        
        weights = base_weight * mask * position_weight
    
    # Common adaptive weighting based on density of missingness
    # Features with more missing values in this batch get higher weights
    feature_missing_density = mask.float().mean(dim=0, keepdim=True)
    density_weight = 1.0 + feature_missing_density * 0.5  # Up to 1.5x for fully missing features
    
    # Apply density weighting
    weights = weights * density_weight
    
    # Apply loss focus - emphasize difficult samples
    focused_loss = element_loss * (1.0 + element_loss)  # Give higher weight to samples with higher loss
    
    # Compute weighted average loss
    weighted_loss = (focused_loss * weights).sum() / (weights.sum() + 1e-8)
    
    return weighted_loss

In [ ]:
def compute_loss(predictions, ground_truth, mask, mechanism=None):
    """
    Compute loss using the enhanced MNAR-weighted loss function.
    """
    return compute_mnar_weighted_loss(predictions, ground_truth, mask, mechanism)

In [ ]:
def compute_rmse(predictions, ground_truth, mask):
    """
    Compute Root Mean Squared Error on masked positions.
    
    Args:
        predictions (torch.Tensor): Predicted values
        ground_truth (torch.Tensor): True values
        mask (torch.Tensor): Binary mask (1 = missing, 0 = present)
        
    Returns:
        float: RMSE value
    """
    masked_preds = predictions[mask == 1]
    masked_truth = ground_truth[mask == 1]
    
    if len(masked_preds) == 0:
        return 0.0
    
    mse = torch.mean((masked_preds - masked_truth) ** 2)
    rmse = torch.sqrt(mse)
    return rmse.item()

In [ ]:
def compute_nrmse(predictions, ground_truth, mask):
    """
    Compute Normalized Root Mean Squared Error on masked positions.
    
    Args:
        predictions (torch.Tensor): Predicted values
        ground_truth (torch.Tensor): True values
        mask (torch.Tensor): Binary mask (1 = missing, 0 = present)
        
    Returns:
        float: NRMSE value
    """
    masked_preds = predictions[mask == 1]
    masked_truth = ground_truth[mask == 1]
    
    if len(masked_preds) == 0:
        return 0.0
    
    mse = torch.mean((masked_preds - masked_truth) ** 2)
    rmse = torch.sqrt(mse)
    
    data_range = ground_truth.max() - ground_truth.min()
    nrmse = rmse / data_range
    return nrmse.item()

In [ ]:
def train_epoch(model, dataloader, optimizer, missing_fraction, mechanisms=None, 
              mechanism_weights=None, scheduler=None, accumulation_steps=4):
    """
    Enhanced training for one epoch with mechanism-weighted sampling and curriculum learning.
    
    Args:
        model (nn.Module): Model to train
        dataloader (DataLoader): Training data loader
        optimizer (Optimizer): Optimizer
        missing_fraction (float): Fraction of values to mask
        mechanisms (list): List of missing data mechanisms to use
        mechanism_weights (dict): Weights for sampling different mechanisms
        scheduler (LRScheduler, optional): Learning rate scheduler
        accumulation_steps (int): Number of steps to accumulate gradients
        
    Returns:
        dict: Dictionary with training metrics
    """
    model.train()
    total_loss = 0
    mechanism_losses = {m: 0.0 for m in mechanisms}
    mechanism_counts = {m: 0 for m in mechanisms}
    
    # Default mechanism weights if not provided
    if mechanism_weights is None:
        mechanism_weights = {m: 1.0 / len(mechanisms) for m in mechanisms}
    
    # Normalize weights to probabilities
    weight_sum = sum(mechanism_weights.values())
    mechanism_probs = {m: w / weight_sum for m, w in mechanism_weights.items()}
    
    # Calculate epoch fraction for curriculum learning
    total_batches = len(dataloader)
    
    # Mixup probability - higher for early training
    mixup_prob = 0.4
    
    if mechanisms is None:
        mechanisms = ["MCAR"]
    
    progress_bar = tqdm(dataloader, desc="Training")
    optimizer.zero_grad()  # Zero gradients at the beginning
    
    for i, batch in enumerate(progress_bar):
        x = batch[0].to(device)
        
        # Curriculum learning for missing fraction
        # Gradually increase difficulty
        epoch_progress = i / total_batches
        if epoch_progress < 0.3:
            # Start with easier task (less missing values)
            curr_missing_fraction = missing_fraction * 0.6
        elif epoch_progress < 0.6:
            # Medium difficulty
            curr_missing_fraction = missing_fraction * 0.8
        else:
            # Full difficulty
            curr_missing_fraction = missing_fraction
        
        # Weighted sampling of mechanisms
        mechanism_choices = list(mechanism_probs.keys())
        mechanism_p = [mechanism_probs[m] for m in mechanism_choices]
        mechanism = np.random.choice(mechanism_choices, p=mechanism_p)
        
        # Create a mask for missing values
        mask = create_missing_mask(x, curr_missing_fraction, mechanism)
        
        # Create input with missing values set to 0
        x_masked = x.clone()
        x_masked[mask == 1] = 0
        
        # Apply mixup with probability
        if np.random.random() < mixup_prob:
            # Create shuffled indices
            indices = torch.randperm(x.size(0), device=device)
            
            # Mix up samples with lambda drawn from beta distribution
            lam = np.random.beta(0.2, 0.2)
            
            # Mix the data
            mixed_x = lam * x_masked + (1 - lam) * x_masked[indices]
            mixed_mask = mask | mask[indices]  # Union of masks
            
            # Forward pass with mixed data
            column_indices = torch.arange(x.shape[1], device=device)
            predictions = model(mixed_x, column_indices, mixed_mask)
            
            # Compute mixed loss
            loss = lam * compute_loss(predictions, x, mask, mechanism) + \
                  (1 - lam) * compute_loss(predictions, x[indices], mask[indices], mechanism)
        else:
            # Standard forward pass
            column_indices = torch.arange(x.shape[1], device=device)
            predictions = model(x_masked, column_indices, mask)
            
            # Compute loss
            loss = compute_loss(predictions, x, mask, mechanism)
        
        # Apply mechanism-specific weight adjustment to loss (beyond just sampling frequency)
        mechanism_loss_weight = 1.0
        if mechanism == "MNAR":
            mechanism_loss_weight = 1.3  # Give MNAR errors more weight
        elif mechanism == "MAR":
            mechanism_loss_weight = 1.1  # Give MAR errors more weight
            
        loss = loss * mechanism_loss_weight
        
        # Track mechanism-specific losses
        mechanism_losses[mechanism] += loss.item()
        mechanism_counts[mechanism] += 1
        
        # Scale loss by accumulation steps   
        loss = loss / accumulation_steps
            
        # Backward pass
        loss.backward()
        
        # Only step optimizer and scheduler after accumulation
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(dataloader):
            # Add gradient clipping (reduced for stability)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            
            if scheduler is not None:
                scheduler.step()
                
            optimizer.zero_grad()
        
        # Update progress bar
        total_loss += loss.item() * accumulation_steps  # Scale back for reporting
        
        # Build mechanism-specific loss reporting
        mech_loss_str = " | ".join(
            [f"{m}: {mechanism_losses[m]/max(1, mechanism_counts[m]):.4f}" 
             for m in mechanisms if mechanism_counts[m] > 0]
        )
        
        progress_bar.set_postfix({
            "loss": loss.item() * accumulation_steps,
            "missing": f"{curr_missing_fraction:.2f}",
            "mech": mechanism,
            "losses": mech_loss_str
        })
    
    # Calculate average loss overall and per mechanism
    avg_loss = total_loss / len(dataloader)
    mech_avg_losses = {
        m: mechanism_losses[m] / max(1, mechanism_counts[m])
        for m in mechanisms
    }
    
    return {
        "loss": avg_loss,
        "mechanism_losses": mech_avg_losses,
        "mechanism_counts": mechanism_counts
    }

In [ ]:
def validate(model, dataloader, missing_fraction, mechanisms=None):
    """
    Validate the model on all mechanisms.
    
    Args:
        model (nn.Module): Model to validate
        dataloader (DataLoader): Validation data loader
        missing_fraction (float): Fraction of values to mask
        mechanisms (list): List of missing data mechanisms to use
        
    Returns:
        dict: Dictionary with validation metrics
    """
    model.eval()
    total_loss = 0
    total_rmse = 0
    total_nrmse = 0
    
    if mechanisms is None:
        mechanisms = ["MCAR"]
    
    mechanism_metrics = {m: {"loss": 0, "rmse": 0, "nrmse": 0, "count": 0} for m in mechanisms}
    
    with torch.no_grad():
        for batch in dataloader:
            x = batch[0].to(device)
            
            for mechanism in mechanisms:
                # Create a mask for missing values
                mask = create_missing_mask(x, missing_fraction, mechanism)
                
                # Create input with missing values set to 0
                x_masked = x.clone()
                x_masked[mask == 1] = 0
                
                # Forward pass
                column_indices = torch.arange(x.shape[1], device=device)
                predictions = model(x_masked, column_indices, mask)
                
                # Compute metrics
                loss = compute_loss(predictions, x, mask)
                rmse = compute_rmse(predictions, x, mask)
                nrmse = compute_nrmse(predictions, x, mask)
                
                # Update mechanism-specific metrics
                mechanism_metrics[mechanism]["loss"] += loss.item()
                mechanism_metrics[mechanism]["rmse"] += rmse
                mechanism_metrics[mechanism]["nrmse"] += nrmse
                mechanism_metrics[mechanism]["count"] += 1
                
                # Update overall metrics
                total_loss += loss.item()
                total_rmse += rmse
                total_nrmse += nrmse
    
    # Calculate averages
    num_evaluations = len(dataloader) * len(mechanisms)
    avg_loss = total_loss / num_evaluations
    avg_rmse = total_rmse / num_evaluations
    avg_nrmse = total_nrmse / num_evaluations
    
    # Calculate mechanism-specific averages
    for m in mechanisms:
        if mechanism_metrics[m]["count"] > 0:
            mechanism_metrics[m]["loss"] /= mechanism_metrics[m]["count"]
            mechanism_metrics[m]["rmse"] /= mechanism_metrics[m]["count"]
            mechanism_metrics[m]["nrmse"] /= mechanism_metrics[m]["count"]
    
    return {
        "loss": avg_loss,
        "rmse": avg_rmse,
        "nrmse": avg_nrmse,
        "mechanisms": mechanism_metrics
    }

In [ ]:
def plot_training_history(history, save_path=None):
    """
    Plot training history.
    
    Args:
        history (dict): Training history
        save_path (str, optional): Path to save the plot
    """
    plt.figure(figsize=(20, 12))
    
    # Plot loss
    plt.subplot(2, 2, 1)
    plt.plot(history["train_loss"], label="Train Loss")
    plt.plot(history["val_loss"], label="Val Loss")
    plt.title("Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    
    # Plot NRMSE
    plt.subplot(2, 2, 2)
    plt.plot(history["val_nrmse"], label="Overall NRMSE")
    plt.plot(history["val_mcar_nrmse"], label="MCAR NRMSE")
    plt.plot(history["val_mar_nrmse"], label="MAR NRMSE")
    plt.plot(history["val_mnar_nrmse"], label="MNAR NRMSE")
    plt.title("NRMSE by Mechanism")
    plt.xlabel("Epoch")
    plt.ylabel("NRMSE")
    plt.legend()
    
    # Plot learning rate
    plt.subplot(2, 2, 3)
    plt.plot(history["lr"])
    plt.title("Learning Rate")
    plt.xlabel("Epoch")
    plt.ylabel("Learning Rate")
    
    if save_path:
        plt.savefig(save_path)
    
    plt.tight_layout()
    plt.show()

In [ ]:
def save_model(model, scaler, config, save_dir="models"):
    """
    Save model and related objects.
    
    Args:
        model (nn.Module): Trained model
        scaler (StandardScaler): Data scaler
        config (dict): Model configuration
        save_dir (str): Directory to save model
    """
    # Create directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    
    # Save model
    model_path = os.path.join(save_dir, "tabular_transformer_relpos.pth")
    torch.save({
        "model_state_dict": model.state_dict(),
        "config": config
    }, model_path)
    
    # Save scaler
    scaler_path = os.path.join(save_dir, "scaler.pkl")
    with open(scaler_path, "wb") as f:
        pickle.dump(scaler, f)
    
    print(f"Model saved to {model_path}")
    print(f"Scaler saved to {scaler_path}")

In [ ]:
def train_model(train_data, val_data, column_indices, config, model_type="single"):
    """
    Train the model (single model or ensemble).
    
    Args:
        train_data (torch.Tensor): Training data
        val_data (torch.Tensor): Validation data
        column_indices (torch.Tensor): Column indices
        config (dict): Configuration dictionary
        model_type (str): Type of model to train ("single" or "ensemble")
        
    Returns:
        tuple: Trained model and training history
    """
    print("Starting model training...")
    
    # Create dataloaders
    train_dataset = TensorDataset(train_data)
    val_dataset = TensorDataset(val_data)
    
    train_dataloader = DataLoader(
        train_dataset, 
        batch_size=config["batch_size"], 
        shuffle=True
    )
    
    val_dataloader = DataLoader(
        val_dataset, 
        batch_size=config["batch_size"]
    )
    
    # Create model based on type
    num_features = train_data.shape[1]
    
    if model_type == "ensemble":
        print("Creating ensemble model with 3 base models...")
        model = EnsembleModel(
            num_features=num_features,
            config=config,
            num_models=3
        ).to(device)
    else:
        print("Creating single transformer model...")
        model = TabularTransformerWithRelPos(
            num_features=num_features,
            d_model=config["d_model"],
            nhead=config["num_heads"],
            num_layers=config["num_layers"],
            dim_feedforward=config["dim_feedforward"],
            dropout=config["dropout"],
            activation=config["activation"],
            max_seq_len=max(2 * num_features, 100)  # Set max_seq_len based on feature count
        ).to(device)
    
    # Set up optimizer and scheduler
    optimizer = optim.AdamW(
        model.parameters(),
        lr=config["learning_rate"],
        weight_decay=config["weight_decay"]
    )
    
    accumulation_steps = config.get("accumulation_steps", 4)
    
    # Learning rate scheduler - use OneCycleLR for better convergence
    total_steps = (len(train_dataloader) // accumulation_steps) * config["num_epochs"]
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=config["learning_rate"],
        total_steps=total_steps,
        pct_start=0.1,  # Shorter warm-up phase
        anneal_strategy='cos',
        div_factor=10.0,  # Less aggressive initial LR reduction
        final_div_factor=1000.0  # Less aggressive final LR
    )
    
    # Training history
    history = {
        "train_loss": [],
        "val_loss": [],
        "val_rmse": [],
        "val_nrmse": [],
        "val_mcar_nrmse": [],
        "val_mar_nrmse": [],
        "val_mnar_nrmse": [],
        "lr": []
    }
    
    # Early stopping
    best_val_loss = float('inf')
    best_model_weights = None
    patience_counter = 0
    
    mechanisms = ["MCAR", "MAR", "MNAR"]
    
    # Get mechanism weights from config if available
    mechanism_weights = config.get("mechanisms_weights", None)
    
    # Training loop
    for epoch in range(config["num_epochs"]):
        print(f"\nEpoch {epoch + 1}/{config['num_epochs']}")
        
        # Train
        train_metrics = train_epoch(
            model,
            train_dataloader,
            optimizer,
            config["missing_fraction"],
            mechanisms=mechanisms,
            mechanism_weights=mechanism_weights,
            scheduler=scheduler,
            accumulation_steps=config.get("accumulation_steps", 4)
        )
        
        # Validate
        val_metrics = validate(
            model,
            val_dataloader,
            config["missing_fraction"],
            mechanisms
        )
        
        # Update history
        history["train_loss"].append(train_metrics["loss"])
        history["val_loss"].append(val_metrics["loss"])
        history["val_rmse"].append(val_metrics["rmse"])
        history["val_nrmse"].append(val_metrics["nrmse"])
        history["val_mcar_nrmse"].append(val_metrics["mechanisms"]["MCAR"]["nrmse"])
        history["val_mar_nrmse"].append(val_metrics["mechanisms"]["MAR"]["nrmse"])
        history["val_mnar_nrmse"].append(val_metrics["mechanisms"]["MNAR"]["nrmse"])
        history["lr"].append(optimizer.param_groups[0]["lr"])
        
        # Print metrics
        print(f"Train Loss: {train_metrics['loss']:.4f}")
        print(f"Val Loss: {val_metrics['loss']:.4f}")
        print(f"Val NRMSE: {val_metrics['nrmse']:.4f}")
        print(f"Val MCAR NRMSE: {val_metrics['mechanisms']['MCAR']['nrmse']:.4f}")
        print(f"Val MAR NRMSE: {val_metrics['mechanisms']['MAR']['nrmse']:.4f}")
        print(f"Val MNAR NRMSE: {val_metrics['mechanisms']['MNAR']['nrmse']:.4f}")
        
        # Check for improvement
        if val_metrics["loss"] < best_val_loss:
            best_val_loss = val_metrics["loss"]
            best_model_weights = model.state_dict().copy()
            patience_counter = 0
            print(f"New best validation loss: {best_val_loss:.4f}")
        else:
            patience_counter += 1
            print(f"No improvement for {patience_counter} epochs")
        
        # Early stopping
        if patience_counter >= config["patience"]:
            print(f"Early stopping after {epoch + 1} epochs")
            break
    
    # Load best model
    if best_model_weights is not None:
        model.load_state_dict(best_model_weights)
    
    return model, history

In [ ]:
def cross_validate_model(data, column_indices, config, n_folds=5, model_type="single"):
    """
    Perform k-fold cross-validation.
    
    Args:
        data (torch.Tensor): Data to split
        column_indices (torch.Tensor): Feature column indices
        config (dict): Configuration dictionary
        n_folds (int): Number of folds
        model_type (str): Type of model to train ("single" or "ensemble")
        
    Returns:
        dict: Dictionary with average metrics across folds
    """
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=SEED)
    
    all_metrics = []
    all_models = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
        print(f"\nTraining fold {fold+1}/{n_folds}")
        
        # Split data
        train_data = data[train_idx]
        val_data = data[val_idx]
        
        # Train model (can be single or ensemble based on model_type)
        model, history = train_model(train_data, val_data, column_indices, config, model_type=model_type)
        
        # Store model
        all_models.append(model)
        
        # Validate
        val_dataloader = DataLoader(TensorDataset(val_data), batch_size=config["batch_size"])
        metrics = validate(model, val_dataloader, config["missing_fraction"], ["MCAR", "MAR", "MNAR"])
        
        all_metrics.append(metrics)
        
        print(f"Fold {fold+1} metrics:")
        print(f"  NRMSE: {metrics['nrmse']:.4f}")
        print(f"  MCAR NRMSE: {metrics['mechanisms']['MCAR']['nrmse']:.4f}")
        print(f"  MAR NRMSE: {metrics['mechanisms']['MAR']['nrmse']:.4f}")
        print(f"  MNAR NRMSE: {metrics['mechanisms']['MNAR']['nrmse']:.4f}")
    
    # Average metrics across folds
    avg_metrics = {
        "nrmse": np.mean([m["nrmse"] for m in all_metrics]),
        "mcar_nrmse": np.mean([m["mechanisms"]["MCAR"]["nrmse"] for m in all_metrics]),
        "mar_nrmse": np.mean([m["mechanisms"]["MAR"]["nrmse"] for m in all_metrics]),
        "mnar_nrmse": np.mean([m["mechanisms"]["MNAR"]["nrmse"] for m in all_metrics])
    }
    
    # Standard deviation of metrics
    std_metrics = {
        "nrmse_std": np.std([m["nrmse"] for m in all_metrics]),
        "mcar_nrmse_std": np.std([m["mechanisms"]["MCAR"]["nrmse"] for m in all_metrics]),
        "mar_nrmse_std": np.std([m["mechanisms"]["MAR"]["nrmse"] for m in all_metrics]),
        "mnar_nrmse_std": np.std([m["mechanisms"]["MNAR"]["nrmse"] for m in all_metrics])
    }
    
    # Add standard deviations to results
    avg_metrics.update(std_metrics)
    
    print("\nAverage metrics across folds:")
    print(f"  NRMSE: {avg_metrics['nrmse']:.4f} ± {avg_metrics['nrmse_std']:.4f}")
    print(f"  MCAR NRMSE: {avg_metrics['mcar_nrmse']:.4f} ± {avg_metrics['mcar_nrmse_std']:.4f}")
    print(f"  MAR NRMSE: {avg_metrics['mar_nrmse']:.4f} ± {avg_metrics['mar_nrmse_std']:.4f}")
    print(f"  MNAR NRMSE: {avg_metrics['mnar_nrmse']:.4f} ± {avg_metrics['mnar_nrmse_std']:.4f}")
    
    return avg_metrics, all_models

In [ ]:
# Main execution code
if __name__ == "__main__":
    # Configuration
    # Optimized configuration for better NRMSE
    config = {
        # Data parameters
        "data_path": "./data/physionet_39_features_only_no_leakage.csv",
        "test_size": 0.2,
        "val_size": 0.1,
        
        # Model parameters - Balanced for overall performance 
        "d_model": 320,      # Increased from 256, but still less than original 384
        "num_heads": 10,     # Increased from 8, but less than original 12  
        "num_layers": 4,     # Reduced from 5
        "dim_feedforward": 1280,  # Increased from 1024
        "dropout": 0.18,     # Slightly decreased from 0.2
        "activation": "gelu",
        
        # Training parameters - Optimized for better generalization
        "batch_size": 48,    # Adjusted from 64
        "learning_rate": 0.00018,  # Slightly decreased
        "weight_decay": 0.012,    # Adjusted from 0.015
        "num_epochs": 100,   # Increased from 80
        "patience": 20,      # Increased from 15
        "missing_fraction": 0.4,  # Increased from 0.35 to focus more on higher missingness
        
        # Learning rate schedule
        "warmup_steps_pct": 0.07,  # Increased from 0.05
        "min_lr_factor": 200.0,    # More gradual decay
        "accumulation_steps": 2,   # Added gradient accumulation for more stable updates
        
        # Save parameters
        "save_dir": "models"
    }
    
    # Create directories with timestamp
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    save_dir = os.path.join(config["save_dir"], f"{current_time}_model-single")
    os.makedirs(save_dir, exist_ok=True)
    
    # Create experiment log file
    experiment_log = os.path.join(save_dir, "experiment_log.txt")
    with open(experiment_log, "w") as f:
        f.write(f"Experiment started at: {current_time}\n")
        f.write("Configuration:\n")
        for key, value in config.items():
            f.write(f"  {key}: {value}\n")
    
    # Load and prepare data
    train_data, val_data, test_data, scaler, column_indices = load_and_prepare_data(
        config["data_path"],
        config["test_size"],
        config["val_size"]
    )
    
    # Set flag to skip k-fold validation
    RUN_KFOLD = False
    
    # Train final model directly (skip K-fold)
    print("\n=== Training Model ===")
    model, history = train_model(train_data, val_data, column_indices, config, model_type="single")
    
    # Plot training history
    plot_training_history(history, os.path.join(save_dir, "training_history.png"))
    
    # Evaluate on test set
    print("\n=== Evaluating on Test Set ===")
    test_dataloader = DataLoader(TensorDataset(test_data), batch_size=config["batch_size"])
    test_metrics = validate(model, test_dataloader, config["missing_fraction"], ["MCAR", "MAR", "MNAR"])
    
    print(f"Test metrics:")
    print(f"  Overall NRMSE: {test_metrics['nrmse']:.4f}")
    print(f"  MCAR NRMSE: {test_metrics['mechanisms']['MCAR']['nrmse']:.4f}")
    print(f"  MAR NRMSE: {test_metrics['mechanisms']['MAR']['nrmse']:.4f}")
    print(f"  MNAR NRMSE: {test_metrics['mechanisms']['MNAR']['nrmse']:.4f}")
    
    # Save test results
    test_results_path = os.path.join(save_dir, "test_results.txt")
    with open(test_results_path, "w") as f:
        f.write("Test Results:\n")
        f.write(f"  Overall NRMSE: {test_metrics['nrmse']:.4f}\n")
        f.write(f"  MCAR NRMSE: {test_metrics['mechanisms']['MCAR']['nrmse']:.4f}\n")
        f.write(f"  MAR NRMSE: {test_metrics['mechanisms']['MAR']['nrmse']:.4f}\n")
        f.write(f"  MNAR NRMSE: {test_metrics['mechanisms']['MNAR']['nrmse']:.4f}\n")
    
    # Save models
    print(f"\n=== Saving Models ===")
    
    # Save final model
    model_save_path = os.path.join(save_dir, "final_model.pth")
    torch.save({
        "model_state_dict": model.state_dict(),
        "config": config,
        "model_type": "single"
    }, model_save_path)
    
    # Save scaler
    scaler_path = os.path.join(save_dir, "scaler.pkl")
    with open(scaler_path, "wb") as f:
        pickle.dump(scaler, f)
    
    # Update experiment log
    with open(experiment_log, "a") as f:
        f.write("\nFinal Test Results:\n")
        f.write(f"  Overall NRMSE: {test_metrics['nrmse']:.4f}\n")
        f.write(f"  MCAR NRMSE: {test_metrics['mechanisms']['MCAR']['nrmse']:.4f}\n")
        f.write(f"  MAR NRMSE: {test_metrics['mechanisms']['MAR']['nrmse']:.4f}\n")
        f.write(f"  MNAR NRMSE: {test_metrics['mechanisms']['MNAR']['nrmse']:.4f}\n")
        
        f.write(f"\nExperiment completed at: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")

    print(f"Training complete! All models and results saved in: {save_dir}")

## Model testing

In [ ]:
def evaluate_imputation_performance(model, test_data, column_indices, missing_percentages=[0.1, 0.2, 0.3, 0.4, 0.5], 
                                    mechanisms=["MCAR", "MAR", "MNAR"]):
    """
    Evaluate the model's imputation performance across different missing percentages and mechanisms.
    
    Args:
        model (nn.Module): Trained model
        test_data (torch.Tensor): Test data tensor
        column_indices (torch.Tensor): Column indices
        missing_percentages (list): List of missing data percentages to evaluate
        mechanisms (list): List of missing data mechanisms to evaluate
        
    Returns:
        pd.DataFrame: DataFrame with NRMSE results for each mechanism and percentage
    """
    model.eval()
    results = {}
    
    with torch.no_grad():
        for mechanism in mechanisms:
            mechanism_results = []
            
            for missing_pct in missing_percentages:
                batch_size = 128
                total_nrmse = 0
                num_batches = 0
                
                # Create DataLoader for test data
                test_dataloader = DataLoader(TensorDataset(test_data), batch_size=batch_size)
                
                for batch in tqdm(test_dataloader, desc=f"Evaluating {mechanism} at {missing_pct*100}%"):
                    x = batch[0].to(device)
                    
                    # Create a mask for missing values
                    mask = create_missing_mask(x, missing_pct, mechanism)
                    
                    # Create input with missing values set to 0
                    x_masked = x.clone()
                    x_masked[mask == 1] = 0
                    
                    # Forward pass
                    predictions = model(x_masked, column_indices, mask)
                    
                    # Compute NRMSE
                    nrmse = compute_nrmse(predictions, x, mask)
                    total_nrmse += nrmse
                    num_batches += 1
                
                avg_nrmse = total_nrmse / num_batches
                mechanism_results.append(avg_nrmse)
                print(f"{mechanism} at {missing_pct*100}% missing: NRMSE = {avg_nrmse:.4f}")
            
            results[mechanism] = mechanism_results
    
    # Create a DataFrame for visualization
    results_df = pd.DataFrame(results, index=[f"{int(pct*100)}%" for pct in missing_percentages])
    results_df.index.name = "Missing Percentage"
    
    return results_df

In [ ]:
def visualize_imputation_performance(results_df, save_path=None):
    """
    Create visualizations for imputation performance.
    
    Args:
        results_df (pd.DataFrame): DataFrame with NRMSE results
        save_path (str, optional): Path to save the figures
    """
    # 1. Create a heatmap visualization
    plt.figure(figsize=(10, 6))
    
    # Create a heatmap with custom colormap (lower values = better = greener)
    sns.heatmap(results_df, annot=True, cmap="RdYlGn_r", fmt=".4f", 
                linewidths=.5, cbar_kws={'label': 'NRMSE (lower is better)'})
    
    plt.title("Imputation Performance (NRMSE) by Missing Mechanism and Percentage")
    plt.tight_layout()
    
    if save_path:
        heatmap_path = save_path.replace(".png", "_heatmap.png")
        plt.savefig(heatmap_path)
        print(f"Heatmap saved to {heatmap_path}")
    
    plt.show()
    
    # 2. Create a line plot
    plt.figure(figsize=(12, 6))
    
    # Convert index to numeric for plotting
    results_df_plot = results_df.copy()
    results_df_plot.index = [int(idx.replace("%", "")) for idx in results_df_plot.index]
    
    # Plot lines for each mechanism
    for column in results_df_plot.columns:
        plt.plot(results_df_plot.index, results_df_plot[column], marker='o', linewidth=2, label=column)
    
    plt.xlabel("Missing Percentage (%)")
    plt.ylabel("NRMSE (lower is better)")
    plt.title("Imputation Performance Across Missing Percentages")
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    plt.xticks(results_df_plot.index)
    
    if save_path:
        line_path = save_path.replace(".png", "_lineplot.png")
        plt.savefig(line_path)
        print(f"Line plot saved to {line_path}")
    
    plt.show()
    
    # 3. Create a bar chart comparison
    plt.figure(figsize=(12, 6))
    
    # Plot grouped bars
    bar_width = 0.25
    r = np.arange(len(results_df_plot.index))
    
    # Plot bars for each mechanism
    for i, column in enumerate(results_df_plot.columns):
        plt.bar(r + i*bar_width, results_df_plot[column], width=bar_width, label=column)
    
    # Add labels and legend
    plt.xlabel("Missing Percentage (%)")
    plt.ylabel("NRMSE (lower is better)")
    plt.title("Imputation Performance by Missing Mechanism and Percentage")
    plt.xticks(r + bar_width, results_df_plot.index)
    plt.legend()
    
    if save_path:
        bar_path = save_path.replace(".png", "_barchart.png")
        plt.savefig(bar_path)
        print(f"Bar chart saved to {bar_path}")
    
    plt.show()
    
    # 4. Create a table visualization
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.axis('off')
    ax.axis('tight')
    
    # Create a table with colored cells based on values
    # First, normalize the data for coloring
    norm_data = results_df.copy()
    for col in norm_data.columns:
        max_val = norm_data[col].max()
        min_val = norm_data[col].min()
        if max_val > min_val:
            norm_data[col] = (norm_data[col] - min_val) / (max_val - min_val)
        else:
            norm_data[col] = 0
    
    # Create a table with cell colors
    cell_colors = plt.cm.RdYlGn_r(norm_data.values)
    table = ax.table(cellText=results_df.values.round(4), 
                    rowLabels=results_df.index,
                    colLabels=results_df.columns,
                    cellColours=cell_colors,
                    loc='center')
    
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1.2, 1.5)
    
    plt.title("NRMSE Values by Missing Mechanism and Percentage", y=0.8)
    
    if save_path:
        table_path = save_path.replace(".png", "_table.png")
        plt.savefig(table_path, bbox_inches='tight')
        print(f"Table visualization saved to {table_path}")
    
    plt.show()
    
    return results_df

In [ ]:
if __name__ == "__main__":
    # Create a section for imputation performance evaluation
    print("\n=== Evaluating Imputation Performance Across Missing Percentages ===")
    
    # Define the missing percentages to evaluate
    missing_percentages = [0.1, 0.2, 0.3, 0.4, 0.5]
    mechanisms = ["MCAR", "MAR", "MNAR"]
    
    # Create a directory for imputation performance visualizations
    imputation_dir = os.path.join(save_dir, "imputation_performance")
    os.makedirs(imputation_dir, exist_ok=True)
    
    # Evaluate imputation performance
    results_df = evaluate_imputation_performance(
        model,                    # Use the final trained model
        test_data,                # Test data
        column_indices,           # Column indices
        missing_percentages,      # Missing percentages
        mechanisms                # Missing mechanisms
    )
    
    # Save the raw results to CSV
    results_csv_path = os.path.join(imputation_dir, "imputation_results.csv")
    results_df.to_csv(results_csv_path)
    print(f"Raw imputation results saved to {results_csv_path}")
    
    # Create and save visualizations
    visualize_imputation_performance(
        results_df,
        save_path=os.path.join(imputation_dir, "imputation_performance.png")
    )
        
    print("\nImputation performance evaluation complete!")

## Impute the original dataset

In [ ]:
def impute_original_dataset(trained_model, scaler, save_path="imputed_original.csv"):
    """
    Load the original CSV dataset, impute missing values using the pre-trained transformer model,
    and save the imputed dataset.
    
    Args:
        trained_model: The pre-trained imputation model
        scaler: The fitted scaler used during model training
        save_path: Path to save the imputed dataset
    """
    print("\n=== Imputing Original Dataset ===")
    
    # Set model to evaluation mode
    trained_model.eval()
    
    # File path
    original_file_path = "./data/physionet_39_features_only_no_leakage.csv"
    
    # Load the original dataset
    print(f"Loading original dataset from {original_file_path}...")
    df_original = pd.read_csv(original_file_path, index_col=None)
    print(f"Original dataset shape: {df_original.shape}")
    
    # Check for missing values
    missing_count = df_original.isna().sum().sum()
    missing_percentage = (missing_count / (df_original.shape[0] * df_original.shape[1])) * 100
    print(f"Dataset contains {missing_count} missing values ({missing_percentage:.2f}% of all values)")
    
    # Create a copy of the original dataset for imputation
    df_imputed = df_original.copy()
    
    # Extract numerical columns for imputation
    numerical_cols = df_original.select_dtypes(include=['number']).columns
    print(f"Found {len(numerical_cols)} numerical columns")
    
    # Create mask for missing values (True where values are missing)
    missing_mask = df_original[numerical_cols].isna()
    
    # Fill missing values with 0 for initial processing
    df_filled = df_original[numerical_cols].fillna(0)
    
    # Scale the data using the provided scaler
    data_scaled = scaler.transform(df_filled)
    
    # Convert to PyTorch tensor
    data_tensor = torch.tensor(data_scaled, dtype=torch.float32).to(device)
    
    # Create column indices tensor
    column_indices = torch.arange(data_tensor.shape[1]).to(device)
    
    # Create mask tensor (1 where values are missing, 0 otherwise)
    mask_tensor = torch.tensor(missing_mask.values, dtype=torch.int).to(device)
    
    print("Performing imputation with trained model...")
    
    with torch.no_grad():
        # Get predictions from the model
        imputed_tensor = trained_model(data_tensor, column_indices, mask_tensor)
        
        # Convert to numpy for processing
        imputed_np = imputed_tensor.cpu().numpy()
        
        # Inverse transform to original scale
        imputed_np = scaler.inverse_transform(imputed_np)
        
        # Create a DataFrame from the imputed values
        imputed_df = pd.DataFrame(imputed_np, columns=numerical_cols, index=df_original.index)
        
        # Replace missing values in the original dataframe with imputed values
        for col in numerical_cols:
            if col in df_imputed.columns:
                missing_idx = df_imputed[col].isna()
                df_imputed.loc[missing_idx, col] = imputed_df.loc[missing_idx, col]
    
    # Save the imputed dataset
    print(f"Saving imputed dataset to {save_path}...")
    df_imputed.to_csv(save_path)
    
    # Verification
    missing_after = df_imputed[numerical_cols].isna().sum().sum()
    print(f"Missing values in numerical columns after imputation: {missing_after}")
    
    total_missing_after = df_imputed.isna().sum().sum()
    if total_missing_after > 0:
        print(f"Total missing values after imputation: {total_missing_after}")
        print("Note: Non-numerical columns may still contain missing values")
    else:
        print("All missing values have been successfully imputed")
    
    print("Imputation complete!")
    
    return df_imputed

In [ ]:
# Execute the imputation process
if __name__ == "__main__":
    print("\n=== Running Imputation on Original Dataset ===")
    
    # Use the already trained model
    # Assuming 'model' and 'scaler' are the trained model and scaler from earlier in the notebook
    imputed_data = impute_original_dataset(model, scaler, "imputed_original.csv")
    
    # Display sample of the imputed data
    print("\nSample of imputed data:")
    print(imputed_data.head())